In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("train.csv")



In [4]:
data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
data = np.array(data)
m, n = data.shape

data_dev = data[:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]

# np.unique(X_train[0])

In [ ]:
def init_params():
    # number of neurons times number of input features (input layer)
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    # number of neurons times number of outputs from the previous layer
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5

    return W1, b1, W2, b2


def ReLU(Z):
    # Element-wise
    return np.maximum(0, Z)


def softmax(Z):
    # Every element in Z is going to be divided by the sum of that element's column.
    A = np.exp(Z) / np.sum(np.exp(Z))

    return A


def forward_prop(W1, b1, W2, b2, X):
    # Z1 is a layer of 10 neurons
    # X (transposed) is a matrix where every column is an intensity of every pixel of a given example

    # W1: (10, 784); X: 784, 41000. Z1: (10, 41000)
    # first row of W1 * first column of X (a row of W1 consists of 784 elements: every element is multiplied by the respective element of X's column, which also consists of 784 elements)
    # first row of W1 * second column of X
    # first row of W1 * third column of X
    # ...
    # first row of W1 * 41000th column of X (that makes up the first row of Z1; it's the first row of W1 multiplied by every column of Z1) (so, every element of Z1 is a sum of 784 elements of W1's first row * 784 elements of X's given column)
    # repeat the same until last row of W1, the 10th row
    # so, every element in the last, 10th row of Z1 is a sum of W1's 10th row * given column of Z1

    # so, the element at the first row, first column of Z1 is a sum of the multiplication of the first weight vector (because every pixel has its own weight) times every pixel of the first example.
    # the element at the first row, second column, is a sum of the multiplication of the first weight vector (because every pixel has its own weight) times every pixel of the second example.

    # In other words, Z[0, 0] is the sum of a vector of weights of a first neuron times the pixels of a first example.


    # Each column in Z1 is a set of pre-activation outputs (scores) for all 10 neurons for a single input example

    # For each example:
    # - Each neuron computes a weighted sum of all 784 pixel values
    # - This score is stored in Z1[i, j]

    # Note: every pixel contributes to 10 neurons, and each neuron has a weight for every pixel.
    # so, there is a weight for every neuron for every pixel

    Z1 = W1.dot(X) + b1

    # Replace all negative values with zeros
    # A1: (10, 41000)
    A1 = ReLU(Z1)

    # W2: (10, 10); A1: (10, 41000); Z2: (10, 41000)

    # Every element in Z2 is a multiplication of a given W2's row times a given column of A1.
    # In other words, Z2[0, 0] is a sum of multiplication of the first neuron's 10 weights times the first column of A1, which, in turn, is a column of values of neurons for Z1 after applying the A1 or the ReLU function in this case.
    Z2 = W2.dot(A1) + b2

    # A2: (10, 41000)
    # Every column is the confidence of neurons that the number (output) is theirs for a given example.
    A2 = softmax(Z2)

    return Z1, A1, Z2, A2


# Y: (m, 1)
def one_hot(Y):
    # one_hot_Y: (m, 10) 
    # 10 in this case because max of Y is 9 (the greatest digit). (we need 10 columns because there are 10 digits)
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))

    # Y.size specifies which row it's accessing; Y specifies which column it's accessing.
    # Iterating over every example using Y.size, and then class label Y as an index, we set the correct class to 1.
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T

    return one_hot_Y


def back_prop(Z1, A1, Z2, A2, W2, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    # Subtraction of actual labels from predictions (for a given example, actual labels are going to be a (10,) column of 9 zeros and 1 one. Predictions are going to be floats.)
    dZ2 = A2 - one_hot_Y
    dW2 = (1 / m) * dZ2 * A1